In [1]:
#Reading CSV files
import pandas as pd
import numpy as np
import csv

def read_csv(file_path):
    lines = []
    
    with open(file_path, "r") as csv_read:
        csvreader = csv.reader(csv_read)
        for row in csvreader:
            lines.append(np.asarray(row))
    csv_read.close()
    #Removing the header and Transposing the data
    lines.pop(0)
    lines = np.array(lines).T
    
    return lines

lines = read_csv("./lens/CogentAnnotationTrain.csv")
print(lines) 

[['Cogent Scanner/1/Colored/20120914164317_L'
  'Cogent Scanner/1/Colored/20120914164317_R'
  'Cogent Scanner/1/Colored/20120914164320_L' ...
  'Cogent Scanner/51/Transparent/20120912143602_R'
  'Cogent Scanner/51/Transparent/20120912143605_L'
  'Cogent Scanner/51/Transparent/20120912143605_R']
 ['Colored' 'Colored' 'Colored' ... 'Transparent' 'Transparent'
  'Transparent']
 ['1' '1' '1' ... '51' '51' '51']
 ...
 ['320' '376' '323' ... '336' '235' '340']
 ['221' '273' '236' ... '189' '196' '196']
 ['130' '125' '129' ... '120' '112' '118']]


In [2]:
#Changing the label classifier to int values
def label_classifier(labels):
    t_labels = {"Colored":0, "Normal":1, "Transparent":2}
    
    for i in range(0, len(labels)):
        labels[i] = t_labels[labels[i]]
        
    return labels.astype(np.int32)

In [3]:
#Getting the images
from PIL import Image
from torchvision.transforms.functional import to_tensor
import time

def read_images(file_path):
    #Read a csv file to know the path of the image
    csv_data = read_csv(file_path)
    
    images_path = csv_data[0]
    labels = label_classifier(csv_data[1])
    
    images = []
    
    for image in images_path:
        img = Image.open("lens/" + image + ".bmp")
        img = img.resize((300, 300))
        images.append(to_tensor(img))
        
    labels = np.array(labels)
    
    ts = int(time.time())
    np.random.seed(ts)
    np.random.shuffle(images)
    np.random.seed(ts)
    np.random.shuffle(labels)
    
    return images, labels

In [4]:
#Setting a variable to work with GPU
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

In [5]:
#Getting the data for the train and test
x_train, y_train = read_images("./lens/CogentAnnotationTrain.csv")
x_test, y_test = read_images("./lens/CogentAnnotationTest.csv")

In [6]:
x_train = torch.stack(x_train)
y_train = torch.from_numpy(y_train)

x_test = torch.stack(x_test)
y_test = torch.from_numpy(y_test)

In [7]:
from torch.utils.data import DataLoader, TensorDataset

train = TensorDataset(x_train, y_train)
test = TensorDataset(x_test, y_test)

train_loaded = DataLoader(train, batch_size=10, shuffle=True)
test_loaded = DataLoader(test, batch_size=10, shuffle=True)

In [8]:
#Getting the pre trained model
from torchvision.models import EfficientNet_B0_Weights, efficientnet_b0
from torch import nn

class LensModel(nn.Module):
    def __init__(self, in_channel=1):
        super(LensModel, self).__init__()
        
        self.weights = EfficientNet_B0_Weights.DEFAULT
        self.model = efficientnet_b0(weights=self.weights)
        self.model.features[0] = nn.Conv2d(1, 32, (3, 3), (2, 2), (1, 1), bias=False)
        
    def forward(self, x):
        return self.model(x)

In [9]:
model = LensModel()
model.to(device)

LensModel(
  (model): EfficientNet(
    (features): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_activation): Sigmoid()
            )
            (2): Conv2dNormActivation(
              (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1

In [10]:
#Model summary
from torchinfo import summary

summary(model=model, input_size=(32, 1, 300, 300))

Layer (type:depth-idx)                                       Output Shape              Param #
LensModel                                                    [32, 1000]                --
├─EfficientNet: 1-1                                          [32, 1000]                --
│    └─Sequential: 2-1                                       [32, 1280, 10, 10]        --
│    │    └─Conv2d: 3-1                                      [32, 32, 150, 150]        288
│    │    └─Sequential: 3-2                                  [32, 16, 150, 150]        1,448
│    │    └─Sequential: 3-3                                  [32, 24, 75, 75]          16,714
│    │    └─Sequential: 3-4                                  [32, 40, 38, 38]          46,640
│    │    └─Sequential: 3-5                                  [32, 80, 19, 19]          242,930
│    │    └─Sequential: 3-6                                  [32, 112, 19, 19]         543,148
│    │    └─Sequential: 3-7                                  [32, 192, 10

In [11]:
#Changing the model to our classes
from torch import nn

model.classifier = nn.Sequential(
    
    
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(1280, 512),
    nn.ReLU(),
    
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(512, 256),
    nn.ReLU(),
    
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(256, 128),
    nn.ReLU(),
    
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(128, 3),
)

model.to(device)

LensModel(
  (model): EfficientNet(
    (features): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_activation): Sigmoid()
            )
            (2): Conv2dNormActivation(
              (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1

In [12]:
#New summary
summary(model, input_size=(32, 1, 300, 300))

Layer (type:depth-idx)                                       Output Shape              Param #
LensModel                                                    [32, 1000]                820,483
├─EfficientNet: 1-1                                          [32, 1000]                --
│    └─Sequential: 2-1                                       [32, 1280, 10, 10]        --
│    │    └─Conv2d: 3-1                                      [32, 32, 150, 150]        288
│    │    └─Sequential: 3-2                                  [32, 16, 150, 150]        1,448
│    │    └─Sequential: 3-3                                  [32, 24, 75, 75]          16,714
│    │    └─Sequential: 3-4                                  [32, 40, 38, 38]          46,640
│    │    └─Sequential: 3-5                                  [32, 80, 19, 19]          242,930
│    │    └─Sequential: 3-6                                  [32, 112, 19, 19]         543,148
│    │    └─Sequential: 3-7                                  [32, 19

In [13]:
#Defining an optimizer and loss
from torch import optim

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [14]:
#Training the model
from tqdm import tqdm
from timeit import default_timer as timer
#Defining the numbers of epochs
epochs = 10

#Lists to get all the data about training
train_loss, test_loss = [], []
accuracy_train, accuracy_test = [], []

start_time = timer()

for epoch in range(epochs):
    total_train_loss = 0
    total_test_loss = 0
    
    model.train()
    
    total = 0
    for index, (image, label) in tqdm(enumerate(train_loaded), desc=f"Fitting Epoch {epoch + 1}"):
        label = label.type(torch.LongTensor)
        image, label = image.to(device), label.to(device)
        
        optimizer.zero_grad()
        
        pred = model(image)
        
        loss = criterion(pred, label)
        total_train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
        pred = nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
    
    train_accuracy = total / len(train)
    total_train_loss = total_train_loss / (index + 1)
    
    accuracy_train.append(train_accuracy)
    train_loss.append(total_train_loss)
    
    #Validating the model
    model.eval()
    total = 0
    for index, (image, label) in tqdm(enumerate(test_loaded), desc="Validating the model"):
        label = label.type(torch.LongTensor)
        image, label = image.to(device), label.to(device)
        pred = model(image)
        
        loss = criterion(pred, label)
        total_test_loss += loss.item()
        
        pred = nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
    test_accuracy = total / len(test)
    total_test_loss = total_test_loss / (index + 1)
    
    accuracy_test.append(test_accuracy)
    test_loss.append(total_test_loss)
    
    print("Epoch: {}/{}  ".format(epoch + 1, epochs),
            "Training loss: {:.4f}  ".format(total_train_loss),
            "Testing loss: {:.4f}  ".format(total_test_loss),
            "Train accuracy: {:.4f}  ".format(train_accuracy),
            "Test accuracy: {:.4f}  ".format(test_accuracy))
    
end_time = timer()
print(f"Time for training: {end_time - start_time:.3f} seconds")

Fitting Epoch 1: 176it [00:49,  3.53it/s]
Validating the model: 176it [00:30,  5.86it/s]


Epoch: 1/10   Training loss: 0.9497   Testing loss: 0.7889   Train accuracy: 0.6691   Test accuracy: 0.6707  


Fitting Epoch 2: 176it [00:50,  3.49it/s]
Validating the model: 176it [00:32,  5.34it/s]


Epoch: 2/10   Training loss: 0.4673   Testing loss: 0.4884   Train accuracy: 0.8021   Test accuracy: 0.8291  


Fitting Epoch 3: 176it [00:50,  3.48it/s]
Validating the model: 176it [00:33,  5.30it/s]


...

Epoch: 8/10   Training loss: 0.1426   Testing loss: 0.3508   Train accuracy: 0.9435   Test accuracy: 0.8900  


Fitting Epoch 9: 176it [01:02,  2.80it/s]
Validating the model: 176it [00:40,  4.34it/s]


Epoch: 9/10   Training loss: 0.0903   Testing loss: 0.3776   Train accuracy: 0.9709   Test accuracy: 0.8900  


Fitting Epoch 10: 176it [01:01,  2.87it/s]
Validating the model: 176it [00:41,  4.20it/s]

Epoch: 10/10   Training loss: 0.0612   Testing loss: 0.4881   Train accuracy: 0.9783   Test accuracy: 0.8758  
Time for training: 902.227 seconds


In [15]:
#Getting the Final Accuracy of the model
import torch.nn.functional as F

model.eval()
t_loss = 0
correct = 0
with torch.no_grad():
    for imgs, labels in tqdm(test_loaded, desc="Getting accuracy"):
        labels = labels.type(torch.LongTensor)
        imgs, labels = imgs.to(device), labels.to(device)
        output = model(imgs)
        t_loss += F.cross_entropy(output, labels, reduction="sum").item()

        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()

t_loss /= len(test_loaded.dataset)
print(
    "Test set: Average loss: {:.4f}, Accuracy: {}/{}, {})\n".format(
        t_loss, correct, len(test_loaded.dataset), 100.0 * correct / len(test_loaded.dataset)
    )
)

Getting accuracy: 100%|███████████████████████████████████████████████████████████████| 176/176 [00:40<00:00,  4.31it/s]

Test set: Average loss: 0.4887, Accuracy: 1537/1755, 87.57834757834758)



In [16]:
#Saving the model and the model parameters
torch.save(model.state_dict(),"../Lens/Model/model.pth")
torch.save(optimizer.state_dict(), "../Lens/Model/optimizer.pth")
print("Model Saved")

Model Saved
